In [ ]:
!python --version

In [ ]:
import os, sys
from scipy import stats
import numpy as np
import pandas as pd

from scipy import stats
is_colab = False

if is_colab:
    # create src and upload stat_lib into it
    from src import stat_lib
else:
    sys.path.insert(1, '../src/')
    import stat_lib

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

### Versão do SciPy

In [ ]:
import scipy

scipy.__version__

### 📌 Wilcoxon signed-rank - stats.wilcoxon()

O **teste de postos sinalizados de Wilcoxon** é um `teste não paramétrico` dual a teste t pareado.

Premissas básicas:

  - Dados pareados
    - Mesmos indivíduos medidos duas vezes  (antes/depois)
  - Diferenças não são normalmente distribuídas
  - Ou amostras pequenas (N < 30)

#### 🧪 Hipótese nula:

𝐻0: mediana das diferenças == 0

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html

In [ ]:
np.random.seed(42)

MU1=140; SSD1=2.5; N1=15

# ruido com muita variancia SD=2
ruido = np.random.normal(loc=0, scale=2, size=N1)

samp1 = np.random.normal(loc=MU1, scale=SSD1, size=N1)

fator = 0.8
samp2 = samp1 * fator + ruido

len(samp1), len(samp2)

### 🧮 Como o teste funciona (passo a passo)

1️⃣ Calcula diferenças:

$d_i = antes_i - depois_i$


2️⃣ Remove diferenças iguais a zero.

3️⃣ Toma valores absolutos:

$d_i = abs(d_i)$

4️⃣ Ranqueia os valores absolutos.

5️⃣ Reaplica os sinais (+ / −).

6️⃣ Soma ranks positivos e negativos.

7️⃣ Estatística W = menor das duas somas.

### As distribuições são nomais? teste de Shapiro-Wilkis

In [ ]:
ret1, text1, text_stat1, stat1, pvalue1 = stat_lib.calc_normalidade_SWT(samp1, 0.05, NS='---')
text1, text_stat1

In [ ]:
ret2, text2, text_stat2, stat2, pvalue2 = stat_lib.calc_normalidade_SWT(samp2, 0.05, NS='---')
text2, text_stat2

### Como o fator = 0.8

In [ ]:
stat, pval = stats.wilcoxon(samp1, samp2, alternative='less')

msg = f"Comparando-se o conjunto amostral com N={N1}, antes e depois do tratamento"
msg += f"\nsegundo o teste de Wilcoxon signed-rank, a estatítica={stat:.2f} e um p-valor={pval:.2e}"
print(msg)

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

colors = ['blue', 'red']
title = ''

for i in range(2):
    color = colors[i]

    if i == 0:
        samples = samp1
        label = 'antes'
        MU = MU1
        SSD = SSD1
    else:
        samples = samp2
        label = 'depois'
        MU = MU1*fator
        SSD = SSD1*fator

    ax = sns.histplot(samples, stat='density', color=color, alpha=0.3, label=label, ax=ax)
    sns.rugplot(samples, color=color, alpha=0.1, ax=ax)

    # max_y = max(p.get_height() for p in ax.patches)

    # Criando o eixo x
    seqx = np.linspace(80, 160, 100)

    # fiting da curva teorica
    normal_pdf = stats.norm.pdf(seqx, MU, SSD)
    sns.lineplot(x=seqx, y=normal_pdf, color=colors[i])

ax.grid()
ax.set_ylabel('probabilidade')
ax.set_xlabel('glicemia (mg/dl)')
ax.set_title(msg);